In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Qualitative Review: Task01 Brain Tumor Segmentation\n",
    "\n",
    "This notebook provides qualitative assessment of trained brain tumor segmentation models.\n",
    "We'll load a trained model, run inference on validation samples, and visualize:\n",
    "\n",
    "- Original multi-modal images\n",
    "- Ground truth segmentations\n",
    "- Model predictions\n",
    "- Probability maps for tumor classes\n",
    "\n",
    "## Setup and Configuration"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import json\n",
    "import sys\n",
    "from pathlib import Path\n",
    "\n",
    "import matplotlib.pyplot as plt\n",
    "import numpy as np\n",
    "import torch\n",
    "from monai.data import decollate_batch\n",
    "from monai.inferers import SlidingWindowInferer\n",
    "from monai.transforms import AsDiscrete\n",
    "\n",
    "# Add project root to path\n",
    "project_root = Path().cwd().parent if Path().cwd().name == 'notebooks' else Path().cwd()\n",
    "sys.path.append(str(project_root))\n",
    "\n",
    "from src.data.loaders_monai import load_monai_decathlon\n",
    "from src.data.transforms_presets import get_transforms_brats_like\n",
    "from src.training.train_enhanced import build_model_from_cfg, get_device\n",
    "\n",
    "print(f\"Project root: {project_root}\")\n",
    "print(f\"CUDA available: {torch.cuda.is_available()}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Configuration\n",
    "MODEL_PATH = \"models/unetr/best.pt\"\n",
    "CONFIG_PATH = \"config/recipes/unetr_multimodal.json\"\n",
    "DATASET_CONFIG_PATH = \"config/datasets/msd_task01_brain.json\"\n",
    "OUTPUT_DIR = \"reports/qualitative\"\n",
    "N_SAMPLES = 2  # Number of validation samples to analyze\n",
    "\n",
    "# Device setup\n",
    "device = get_device(\"auto\")\n",
    "print(f\"Using device: {device}\")\n",
    "\n",
    "# Create output directory\n",
    "output_dir = Path(OUTPUT_DIR)\n",
    "output_dir.mkdir(parents=True, exist_ok=True)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Load Configuration and Dataset"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load configurations\n",
    "with open(CONFIG_PATH, 'r') as f:\n",
    "    config = json.load(f)\n",
    "    \n",
    "with open(DATASET_CONFIG_PATH, 'r') as f:\n",
    "    dataset_config = json.load(f)\n",
    "\n",
    "print(\"Model config:\")\n",
    "print(f\"  Architecture: {config.get('model', {}).get('arch', 'unetr')}\")\n",
    "print(f\"  Image size: {config.get('model', {}).get('img_size', [128, 128, 128])}\")\n",
    "print(f\"  Output channels: {config.get('model', {}).get('out_channels', 2)}\")\n",
    "\n",
    "print(\"\\nDataset config:\")\n",
    "print(f\"  Dataset: {dataset_config.get('dataset_id', 'Task01_BrainTumour')}\")\n",
    "print(f\"  Transforms: {dataset_config.get('transforms', 'brats_like')}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Setup transforms and load validation data\n",
    "spacing = tuple(dataset_config.get(\"spacing\", (1.0, 1.0, 1.0)))\n",
    "transforms_train, transforms_val = get_transforms_brats_like(spacing=spacing)\n",
    "\n",
    "# Load validation dataset\n",
    "val_data = load_monai_decathlon(\n",
    "    root_dir=\"data/msd\",\n",
    "    task=\"Task01_BrainTumour\",\n",
    "    section=\"validation\",\n",
    "    download=True,\n",
    "    cache_rate=0.0,\n",
    "    num_workers=2,\n",
    "    transform=transforms_val,\n",
    "    batch_size=1,\n",
    "    pin_memory=False,\n",
    ")\n",
    "\n",
    "val_loader = val_data[\"dataloader\"]\n",
    "print(f\"Validation dataset loaded with {len(val_loader)} samples\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Load Trained Model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Determine model parameters from validation data\n",
    "sample_batch = next(iter(val_loader))\n",
    "sample_image = sample_batch[\"image\"]\n",
    "in_channels = sample_image.shape[1]\n",
    "out_channels = config.get(\"model\", {}).get(\"out_channels\", 2)\n",
    "\n",
    "print(f\"Input channels: {in_channels}\")\n",
    "print(f\"Output channels: {out_channels}\")\n",
    "print(f\"Sample image shape: {sample_image.shape}\")\n",
    "\n",
    "# Build and load model\n",
    "model = build_model_from_cfg(config, in_channels, out_channels).to(device)\n",
    "\n",
    "# Load trained weights\n",
    "if Path(MODEL_PATH).exists():\n",
    "    checkpoint = torch.load(MODEL_PATH, map_location=device)\n",
    "    if \"model\" in checkpoint:\n",
    "        model.load_state_dict(checkpoint[\"model\"])\n",
    "    else:\n",
    "        model.load_state_dict(checkpoint)\n",
    "    print(f\"Model loaded from {MODEL_PATH}\")\n",
    "else:\n",
    "    print(f\"Warning: Model file not found at {MODEL_PATH}\")\n",
    "    print(\"Using randomly initialized model for demonstration\")\n",
    "\n",
    "model.eval()\n",
    "print(\"Model ready for inference\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Setup Inference Components"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Setup sliding window inferer\n",
    "roi_size = tuple(config.get(\"model\", {}).get(\"img_size\", [128, 128, 128]))\n",
    "inferer = SlidingWindowInferer(\n",
    "    roi_size=roi_size,\n",
    "    sw_batch_size=1,\n",
    "    overlap=0.25,\n",
    ")\n",
    "\n",
    "# Setup post-processing transforms\n",
    "post_pred = AsDiscrete(argmax=True, to_onehot=out_channels)\n",
    "post_label = AsDiscrete(to_onehot=out_channels)\n",
    "\n",
    "print(f\"Sliding window ROI size: {roi_size}\")\n",
    "print(f\"Inference setup complete\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Visualization Functions"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def plot_multi_slice_comparison(image, gt_mask, pred_mask, prob_map, case_name, save_path=None):\n",
    "    \"\"\"\n",
    "    Plot multi-slice comparison showing image, ground truth, prediction, and probability map.\n",
    "    \n",
    "    Args:\n",
    "        image: (C, H, W, D) input image tensor\n",
    "        gt_mask: (H, W, D) ground truth mask\n",
    "        pred_mask: (H, W, D) prediction mask\n",
    "        prob_map: (H, W, D) probability map for tumor class\n",
    "        case_name: Case identifier for title\n",
    "        save_path: Optional path to save the figure\n",
    "    \"\"\"\n",
    "    # Use first channel as background (T1)\n",
    "    img = image[0].cpu().numpy()\n",
    "    gt = gt_mask.cpu().numpy()\n",
    "    pred = pred_mask.cpu().numpy()\n",
    "    prob = prob_map.cpu().numpy()\n",
    "    \n",
    "    # Select slices (25%, 50%, 75% depth)\n",
    "    _, _, D = img.shape\n",
    "    slices = [D // 4, D // 2, 3 * D // 4]\n",
    "    \n",
    "    fig, axes = plt.subplots(4, 3, figsize=(15, 16))\n",
    "    \n",
    "    for i, z in enumerate(slices):\n",
    "        # Normalize image slice\n",
    "        img_slice = img[..., z]\n",
    "        img_slice = (img_slice - img_slice.min()) / (img_slice.max() - img_slice.min() + 1e-8)\n",
    "        \n",
    "        gt_slice = gt[..., z]\n",
    "        pred_slice = pred[..., z]\n",
    "        prob_slice = prob[..., z]\n",
    "        \n",
    "        # Row 0: Original image\n",
    "        axes[0, i].imshow(img_slice, cmap='gray')\n",
    "        axes[0, i].set_title(f'T1 Image - Slice {z}/{D-1}')\n",
    "        axes[0, i].axis('off')\n",
    "        \n",
    "        # Row 1: Ground truth overlay\n",
    "        axes[1, i].imshow(img_slice, cmap='gray')\n",
    "        axes[1, i].imshow(np.ma.masked_where(gt_slice == 0, gt_slice), \n",
    "                         cmap='Greens', alpha=0.5)\n",
    "        axes[1, i].set_title(f'Ground Truth - Slice {z}')\n",
    "        axes[1, i].axis('off')\n",
    "        \n",
    "        # Row 2: Prediction overlay\n",
    "        axes[2, i].imshow(img_slice, cmap='gray')\n",
    "        axes[2, i].imshow(np.ma.masked_where(pred_slice == 0, pred_slice), \n",
    "                         cmap='Reds', alpha=0.5)\n",
    "        axes[2, i].set_title(f'Prediction - Slice {z}')\n",
    "        axes[2, i].axis('off')\n",
    "        \n",
    "        # Row 3: Probability map\n",
    "        axes[3, i].imshow(img_slice, cmap='gray')\n",
    "        prob_overlay = axes[3, i].imshow(\n",
    "            np.ma.masked_where(prob_slice < 0.1, prob_slice), \n",
    "            cmap='hot', alpha=0.7, vmin=0, vmax=1\n",
    "        )\n",
    "        axes[3, i].set_title(f'Probability Map - Slice {z}')\n",
    "        axes[3, i].axis('off')\n",
    "    \n",
    "    # Add colorbar for probability map\n",
    "    cbar = plt.colorbar(prob_overlay, ax=axes[3, :], orientation='horizontal', \n",
    "                       fraction=0.05, pad=0.1)\n",
    "    cbar.set_label('Tumor Probability')\n",
    "    \n",
    "    plt.suptitle(f'Qualitative Analysis: {case_name}', fontsize=16, y=0.98)\n",
    "    plt.tight_layout()\n",
    "    \n",
    "    if save_path:\n",
    "        plt.savefig(save_path, dpi=150, bbox_inches='tight')\n",
    "        print(f\"Figure saved to: {save_path}\")\n",
    "    \n",
    "    plt.show()\n",
    "\n",
    "def calculate_dice_score(pred, gt):\n",
    "    \"\"\"Calculate Dice coefficient between prediction and ground truth.\"\"\"\n",
    "    pred_flat = pred.flatten()\n",
    "    gt_flat = gt.flatten()\n",
    "    \n",
    "    intersection = (pred_flat * gt_flat).sum()\n",
    "    total = pred_flat.sum() + gt_flat.sum()\n",
    "    \n",
    "    if total == 0:\n",
    "        return 1.0  # Both empty\n",
    "    \n",
    "    dice = (2.0 * intersection) / total\n",
    "    return dice"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Run Qualitative Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Run inference on validation samples\n",
    "results = []\n",
    "\n",
    "with torch.no_grad():\n",
    "    for i, batch in enumerate(val_loader):\n",
    "        if i >= N_SAMPLES:\n",
    "            break\n",
    "            \n",
    "        print(f\"\\n=== Processing sample {i+1}/{N_SAMPLES} ===\")\n",
    "        \n",
    "        # Get batch data\n",
    "        images = batch[\"image\"].to(device)  # (1, C, H, W, D)\n",
    "        labels = batch[\"label\"].to(device)  # (1, 1, H, W, D)\n",
    "        \n",
    "        # Run inference\n",
    "        logits = inferer(images, model)  # (1, out_channels, H, W, D)\n",
    "        \n",
    "        # Get probabilities and predictions\n",
    "        probs = torch.softmax(logits, dim=1)\n",
    "        \n",
    "        # Post-process predictions and labels\n",
    "        pred_list = [post_pred(p) for p in decollate_batch(logits)]\n",
    "        gt_list = [post_label(l) for l in decollate_batch(labels)]\n",
    "        \n",
    "        # Extract tensors (remove batch dimension)\n",
    "        image_tensor = images[0]  # (C, H, W, D)\n",
    "        pred_tensor = pred_list[0]  # (out_channels, H, W, D)\n",
    "        gt_tensor = gt_list[0]  # (out_channels, H, W, D)\n",
    "        prob_tensor = probs[0]  # (out_channels, H, W, D)\n",
    "        \n",
    "        # Get class 1 (tumor) masks and probabilities\n",
    "        if out_channels > 1:\n",
    "            pred_mask = pred_tensor[1]  # (H, W, D)\n",
    "            gt_mask = gt_tensor[1]  # (H, W, D)\n",
    "            prob_map = prob_tensor[1]  # (H, W, D)\n",
    "        else:\n",
    "            pred_mask = pred_tensor[0]\n",
    "            gt_mask = gt_tensor[0]\n",
    "            prob_map = prob_tensor[0]\n",
    "        \n",
    "        # Calculate metrics\n",
    "        dice_score = calculate_dice_score(\n",
    "            pred_mask.cpu().numpy(), \n",
    "            gt_mask.cpu().numpy()\n",
    "        )\n",
    "        \n",
    "        # Calculate volume statistics\n",
    "        gt_volume = gt_mask.sum().item()\n",
    "        pred_volume = pred_mask.sum().item()\n",
    "        max_prob = prob_map.max().item()\n",
    "        mean_prob = prob_map.mean().item()\n",
    "        \n",
    "        print(f\"Dice Score: {dice_score:.4f}\")\n",
    "        print(f\"GT Volume: {gt_volume:.0f} voxels\")\n",
    "        print(f\"Pred Volume: {pred_volume:.0f} voxels\")\n",
    "        print(f\"Max Probability: {max_prob:.4f}\")\n",
    "        print(f\"Mean Probability: {mean_prob:.4f}\")\n",
    "        \n",
    "        # Store results\n",
    "        case_name = f\"Case_{i+1:02d}\"\n",
    "        results.append({\n",
    "            'case_name': case_name,\n",
    "            'dice_score': dice_score,\n",
    "            'gt_volume': gt_volume,\n",
    "            'pred_volume': pred_volume,\n",
    "            'max_prob': max_prob,\n",
    "            'mean_prob': mean_prob,\n",
    "        })\n",
    "        \n",
    "        # Create visualization\n",
    "        save_path = output_dir / f\"{case_name}_qualitative_analysis.png\"\n",
    "        plot_multi_slice_comparison(\n",
    "            image_tensor, gt_mask, pred_mask, prob_map, \n",
    "            case_name, save_path\n",
    "        )"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Summary Statistics"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Calculate summary statistics\n",
    "if results:\n",
    "    dice_scores = [r['dice_score'] for r in results]\n",
    "    mean_dice = np.mean(dice_scores)\n",
    "    std_dice = np.std(dice_scores)\n",
    "    \n",
    "    print(\"\\n=== SUMMARY STATISTICS ===\")\n",
    "    print(f\"Number of cases analyzed: {len(results)}\")\n",
    "    print(f\"Mean Dice Score: {mean_dice:.4f} ± {std_dice:.4f}\")\n",
    "    print(f\"Min Dice Score: {min(dice_scores):.4f}\")\n",
    "    print(f\"Max Dice Score: {max(dice_scores):.4f}\")\n",
    "    \n",
    "    # Create summary table\n",
    "    print(\"\\n=== DETAILED RESULTS ===\")\n",
    "    print(f\"{'Case':<8} {'Dice':<8} {'GT Vol':<8} {'Pred Vol':<10} {'Max Prob':<10} {'Mean Prob':<10}\")\n",
    "    print(\"-\" * 60)\n",
    "    for r in results:\n",
    "        print(f\"{r['case_name']:<8} {r['dice_score']:<8.4f} {r['gt_volume']:<8.0f} \"\n",
    "              f\"{r['pred_volume']:<10.0f} {r['max_prob']:<10.4f} {r['mean_prob']:<10.4f}\")\n",
    "    \n",
    "    # Save results to JSON\n",
    "    results_file = output_dir / \"qualitative_analysis_results.json\"\n",
    "    with open(results_file, 'w') as f:\n",
    "        json.dump({\n",
    "            'summary': {\n",
    "                'n_cases': len(results),\n",
    "                'mean_dice': mean_dice,\n",
    "                'std_dice': std_dice,\n",
    "                'min_dice': min(dice_scores),\n",
    "                'max_dice': max(dice_scores),\n",
    "            },\n",
    "            'cases': results\n",
    "        }, f, indent=2)\n",
    "    \n",
    "    print(f\"\\nResults saved to: {results_file}\")\n",
    "else:\n",
    "    print(\"No results to summarize.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Conclusion\n",
    "\n",
    "This qualitative analysis provides insights into the model's learned behavior:\n",
    "\n",
    "1. **Segmentation Quality**: The Dice scores indicate overall segmentation accuracy\n",
    "2. **Spatial Consistency**: Multi-slice views show how well the model maintains spatial coherence\n",
    "3. **Confidence Assessment**: Probability maps reveal model uncertainty and confidence regions\n",
    "4. **Volume Estimation**: Comparison of predicted vs ground truth volumes shows systematic biases\n",
    "\n",
    "The visualizations are saved in the `reports/qualitative/` directory for further analysis and comparison across different model configurations."
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}